In [1]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


from plyfile import PlyData, PlyElement
import numpy as np


In [2]:
init_notebook_mode(connected=True)

### Gather the xyz points from the ply file

In [3]:
plydata = PlyData.read('ply/cow2.ply')

nr_points=plydata.elements[0].count
nr_faces=plydata.elements[1].count

# unpack vertices to array
vertices_pt_xyz =np.array([np.array(plydata['vertex'][k]) for k in range(nr_points)])

vertices_pt_xyz=np.array(map(list, vertices_pt_xyz))
vertices_pt_xyz[:,2] -= 5 # down the z axis
x,y,z=zip(*vertices_pt_xyz)

# unpack faces to array
faces_pt_3v = np.array([np.array(plydata['face'][k][0]) for k in range(nr_faces)])
faces_pt_3v[0]

array([0, 1, 2], dtype=int32)

### Plot!

In [4]:

cowScat =Scatter3d(x=x,
                y=y,
                z=z,
                mode='markers',
                marker=dict(
                    size=2,
                    color=z,                # set color to an array/list of desired values
                    colorscale='Red',   # choose a colorscale
                    opacity=0.8)
               )

xMark =Scatter3d(x=[5],
            y=[0],
            z=[0],
            mode='markers',
            marker=dict(size=15,color='rgb(255,0,0)')#'#f44242')
           )

yMark =Scatter3d(x=[0],
            y=[5],
            z=[0],
            mode='markers',
            marker=dict(size=15,color='rgb(0,255,0)')
           )

zMark =Scatter3d(x=[0],
            y=[0],
            z=[-5],
            mode='markers',
            marker=dict(size=15,color='rgb(0,0,255)')
           )

        
axis=dict(showbackground=False,
          showline=True,  
          zeroline=True,
          showgrid=True,
          showticklabels=True,
          title='' 
          )

layout = Layout(title="PLY", 
                width=800,
                height=800,
                showlegend=False,
                scene=Scene(aspectmode='manual',
                            aspectratio=dict(x=1, y=1, z=1),
                            xaxis=dict(range=[-10, 10], title='x Axis'),
                            yaxis=dict(range=[-10, 10], title='y Axis'),
                            zaxis=dict(range=[-20,0], title='z Axis'),
                            camera = dict(
                                up=dict(x=1, y=0, z=0),
                                center=dict(x=0, y=0, z=0),
                                eye=dict(x=0, y=0, z=2)
                            ),

                           ),
                margin=Margin(t=100),
                hovermode='closest',
                
                )

data=Data([cowScat,xMark,yMark,zMark])
fig=Figure(data=data, layout=layout)
iplot(fig)

### Parameters of the frustum 

In [5]:
fovy = 90.0 * (np.pi/180)
aspect = 1.0;
fovx = fovy * aspect;
nearDist = 0.1;
farDist = 10.0;

right = nearDist * np.tan(fovx/2.0)
left  = nearDist * -np.tan(fovx/2.0)

top      = nearDist * np.tan(fovy/2.0)
bottom   = nearDist * -np.tan(fovy/2.0)

yScale = 1.0 / np.tan(fovy / 2);
xScale = yScale/aspect;
nearmfar = nearDist - farDist;

### The test

In [6]:
# Define a gaze direction
gazeVec_xyz = [.05,.4,-1] 
gazeDir_xyz = gazeVec_xyz  / np.linalg.norm(gazeVec_xyz)

# Create two points along the gaze direction
nearGazePt_xyz = gazeDir_xyz * nearDist
farGazePt_xyz = gazeDir_xyz * 5

projMat_4x4 = np.zeros([4,4]);
projMat_4x4[0,0] = xScale
projMat_4x4[1,1] = yScale
projMat_4x4[2,2] = (farDist+nearDist) / (nearDist-farDist);
projMat_4x4[2,3] = -1.0;
projMat_4x4[3,2] = 2*(nearDist * farDist) / (nearDist-farDist)
projMat_4x4[3,3] = 1

def projectVertex(xyzw,projMatIn_4x4):
    dataOutXYZW = np.dot( projMatIn_4x4,np.transpose(xyzw))
    dataOutXYZ = dataOutXYZW / dataOutXYZW[3]
    return dataOutXYZW[0:3]

# Project the two points and spit out their coordinates
n_xyz = projectVertex(np.hstack([nearGazePt_xyz,1]),projMat_4x4)
f_xyz = projectVertex(np.hstack([farGazePt_xyz,1]),projMat_4x4)

print '\nNear: %1.2f, %1.2f, %1.2f' % (n_xyz[0], n_xyz[1], n_xyz[2])
print ' Far: %1.2f, %1.2f, %1.2f' % (f_xyz[0], f_xyz[1], f_xyz[2])


Near: 0.00, 0.04, -0.91
 Far: 0.23, 1.85, 3.73


### Sadly, the two-point test is failing me

# https://www.khronos.org/registry/OpenGL-Refpages/gl2.1/xhtml/glFrustum.xml

http://www.songho.ca/opengl/gl_transform.html